In [24]:
from itertools import product

import pandas as pd
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from umap import UMAP

In [25]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [26]:
df = pd.read_csv('data/hf_patients_data.csv')
df.drop(columns=['id_global_pat'], inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73449 entries, 0 to 73449
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   los       73449 non-null  int64  
 1   Provider  73449 non-null  object 
 2   female    73449 non-null  float64
 3   age       73449 non-null  int64  
 4   comI10    73449 non-null  int64  
 5   comI20    73449 non-null  int64  
 6   comI25    73449 non-null  int64  
 7   comI48    73449 non-null  int64  
 8   comI60    73449 non-null  int64  
 9   comJ44    73449 non-null  int64  
 10  comE10    73449 non-null  int64  
 11  comN18    73449 non-null  int64  
 12  comE66    73449 non-null  int64  
 13  comI21    73449 non-null  int64  
 14  comI05    73449 non-null  int64  
 15  comI34    73449 non-null  int64  
 16  comI42    73449 non-null  int64  
 17  comN17    73449 non-null  int64  
 18  comC00    73449 non-null  int64  
 19  comI60_9  73449 non-null  int64  
 20  comF00    73449 non-null  int64  

In [27]:
# Для тестов
# df = df.iloc[:100, :]
# df.info()

In [28]:
num_cols = list(df.select_dtypes(include='number').columns)
num_cols

['los',
 'female',
 'age',
 'comI10',
 'comI20',
 'comI25',
 'comI48',
 'comI60',
 'comJ44',
 'comE10',
 'comN18',
 'comE66',
 'comI21',
 'comI05',
 'comI34',
 'comI42',
 'comN17',
 'comC00',
 'comI60_9',
 'comF00',
 'comU07']

In [29]:
# Отмасштабируем данные
scaler = StandardScaler().set_output(transform='pandas')
df_scaled = scaler.fit_transform(df[num_cols])

# Преобразуем пол в категориальный тип данных
df_scaled.head()

,los,female,age,comI10,comI20,comI25,comI48,comI60,comJ44,comE10,comN18,comE66,comI21,comI05,comI34,comI42,comN17,comC00,comI60_9,comF00,comU07
0,2.011169,-1.202200,0.642192,-2.002283,-2.022729,-0.406233,-0.588519,-0.238338,-0.346668,-0.553816,-0.149799,-0.350745,-0.229434,-0.101228,-0.192177,-0.183277,-0.060967,-0.364021,-1.354256,-0.020877,-0.560509
1,-0.914888,-1.202200,1.633415,0.499430,0.494382,-0.406233,-0.588519,-0.238338,-0.346668,-0.553816,-0.149799,-0.350745,-0.229434,-0.101228,-0.192177,-0.183277,-0.060967,-0.364021,0.738413,-0.020877,-0.560509
2,1.150564,0.831809,1.303007,-2.002283,0.494382,-0.406233,-0.588519,-0.238338,-0.346668,1.805653,-0.149799,-0.350745,-0.229434,-0.101228,-0.192177,-0.183277,-0.060967,-0.364021,0.738413,-0.020877,-0.560509
3,0.634201,0.831809,-0.018624,0.499430,0.494382,-0.406233,1.699182,-0.238338,2.884604,-0.553816,-0.149799,-0.350745,-0.229434,-0.101228,5.203541,-0.183277,-0.060967,-0.364021,0.738413,-0.020877,-0.560509
4,-0.914888,-1.202200,0.229182,0.499430,0.494382,-0.406233,-0.588519,-0.238338,-0.346668,-0.553816,-0.149799,-0.350745,-0.229434,-0.101228,-0.192177,-0.183277,-0.060967,-0.364021,-1.354256,-0.020877,-0.560509


In [40]:
# Подготовим столбцы с координатами UMAP для двух вариантов понижения размерности

umap = UMAP(metric='hamming', n_neighbors=150, min_dist=0.2, n_components=2, 
                random_state=42, n_jobs=1)
embedding = umap.fit_transform(df_scaled)
df['x1'] = embedding[:, 0]
df['y1'] = embedding[:, 1]

umap = UMAP(metric='hamming', n_neighbors=25, min_dist=0.5, n_components=2, 
                random_state=42, n_jobs=1)
embedding = umap.fit_transform(df_scaled)
df['x2'] = embedding[:, 0]
df['y2'] = embedding[:, 1]

In [31]:
def draw_clusters_original_data(n_clusters, init, log_dict):
    '''
        f
    '''

    # Copy of original dataset
    df_cp = df.copy()

    # Clustering
    km = KMeans(n_clusters=n_clusters, init=init, max_iter=1000, algorithm='lloyd', random_state=42)
    km.fit(df_scaled)
    df_cp['cluster'] = km.labels_
    
    key = f'Kmeans on original data with n_clusters={n_clusters}, init={init}'
    
    try:
        sil = silhouette_score(df_scaled, km.labels_)
        chs = calinski_harabasz_score(df_scaled, km.labels_)
        dbs = davies_bouldin_score(df_scaled, km.labels_)

        sil = round(sil, 4)   # Silhoulette
        chs = round(chs, 4)    # Calinski Harabasz score
        dbs = round(dbs, 4)    # Davies Bouldin score
    except Exception as e:
        sil = 'Error'
        chs = 'Error'
        dbs = 'Error'

    log_dict[key] = (sil, chs, dbs)

    # First Umap plot
    plot_title = f'First Umap plot with KMeans n_clusters={n_clusters}, init={init}'

    fig = px.scatter(df_cp, 'x1', 'y1', color='cluster', labels={'color': 'female'}, title=plot_title,
                    width=1200, height=1000, template='simple_white', 
                    hover_data=num_cols)
    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50), paper_bgcolor="White")
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.write_html(f'./kmeans_plots/original_data/First_UMAP_KMeans_n_clusters_{n_clusters}_init_{init}.html')

    # Second Umap plot
    plot_title = f'Second Umap plot with KMeans n_clusters={n_clusters}, init={init}'

    fig = px.scatter(df_cp, 'x2', 'y2', color='cluster', labels={'color': 'female'}, title=plot_title,
                    width=1200, height=1000, template='simple_white', 
                    hover_data=num_cols)
    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50), paper_bgcolor="White")
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.write_html(f'./kmeans_plots/original_data/Second_UMAP_KMeans_n_clusters_{n_clusters}_init_{init}.html')

In [32]:
def draw_clusters_umapped_data(n_clusters, init, log_dict):
    '''
        f
    '''

    # Copy of original dataset
    df_cp = df.copy()

    # Clustering 1
    km = KMeans(n_clusters=n_clusters, init=init, max_iter=1000, algorithm='lloyd', random_state=42)
    km.fit(df_cp[['x1', 'y1']])
    df_cp['cluster'] = km.labels_
    
    key = f'Kmeans on umapped coords 1 with n_clusters={n_clusters}, init={init}'
    
    try:
        sil = silhouette_score(df_cp[['x1', 'y1']], km.labels_)
        chs = calinski_harabasz_score(df_cp[['x1', 'y1']], km.labels_)
        dbs = davies_bouldin_score(df_cp[['x1', 'y1']], km.labels_)

        sil = round(sil, 4)   # Silhoulette
        chs = round(chs, 4)    # Calinski Harabasz score
        dbs = round(dbs, 4)    # Davies Bouldin score
    except Exception as e:
        sil = 'Error'
        chs = 'Error'
        dbs = 'Error'

    log_dict[key] = (sil, chs, dbs)

    # First Umap plot
    plot_title = f'First Umap plot with KMeans n_clusters={n_clusters}, init={init}'

    fig = px.scatter(df_cp, 'x1', 'y1', color='cluster', labels={'color': 'female'}, title=plot_title,
                    width=1200, height=1000, template='simple_white', 
                    hover_data=num_cols)
    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50), paper_bgcolor="White")
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.write_html(f'./kmeans_plots/umapped_data/First_UMAP_KMeans_n_clusters_{n_clusters}_init_{init}.html')


    # Clustering 2
    km = KMeans(n_clusters=n_clusters, init=init, max_iter=1000, algorithm='lloyd', random_state=42)
    km.fit(df_cp[['x2', 'y2']])
    df_cp['cluster'] = km.labels_

    key = f'Kmeans on umapped coords 2 with n_clusters={n_clusters}, init={init}'
    
    try:
        sil = silhouette_score(df_cp[['x2', 'y2']], km.labels_)
        chs = calinski_harabasz_score(df_cp[['x2', 'y2']], km.labels_)
        dbs = davies_bouldin_score(df_cp[['x2', 'y2']], km.labels_)

        sil = round(sil, 4)   # Silhoulette
        chs = round(chs, 4)    # Calinski Harabasz score
        dbs = round(dbs, 4)    # Davies Bouldin score
    except Exception as e:
        sil = 'Error'
        chs = 'Error'
        dbs = 'Error'

    log_dict[key] = (sil, chs, dbs)

    # Second Umap plot
    plot_title = f'Second Umap plot with KMeans n_clusters={n_clusters}, init={init}'

    fig = px.scatter(df_cp, 'x2', 'y2', color='cluster', labels={'color': 'female'}, title=plot_title,
                    width=1200, height=1000, template='simple_white', 
                    hover_data=num_cols)
    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50), paper_bgcolor="White")
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.write_html(f'./kmeans_plots/umapped_data/Second_UMAP_KMeans_n_clusters_{n_clusters}_init_{init}.html')

In [33]:
log_dict = {}

In [34]:
n_clusters_list = list(range(4, 10, 1))
init_list = ['k-means++', 'random']

params = list(product(n_clusters_list, init_list))

for n_clusters, init in tqdm(params, total=len(params)):
    draw_clusters_original_data(n_clusters, init, log_dict)
    draw_clusters_umapped_data(n_clusters, init, log_dict)


100%|██████████| 12/12 [15:12<00:00, 76.03s/it]


In [35]:
df_metrics = pd.DataFrame(log_dict).T.rename(columns={0: 'Silhoulette score', 1: 'Calinski Harabasz score', 2: 'Davies Bouldin score'})
df_metrics.to_excel('./kmeans_plots/kmeans_metrics.xlsx')

,los,Provider,female,age,comI10,comI20,comI25,comI48,comI60,comJ44,comE10,comN18,comE66,comI21,comI05,comI34,comI42,comN17,comC00,comI60_9,comF00,comU07
0,22,"СПб ГБУЗ ""Госпиталь для ветеранов войн""",0.0,82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,"СПб ГБУЗ ""Городская Покровская больница""",0.0,94,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,17,"СПб ГБУЗ ""Госпиталь для ветеранов войн""",1.0,90,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,14,"СПб ГБУЗ ""Городская Покровская больница""",1.0,74,1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0
4,5,"СПб ГБУЗ ""Городская Покровская больница""",0.0,77,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
km = KMeans(n_clusters=5, init='k-means++', max_iter=1000, algorithm='lloyd', random_state=42)
km.fit(df[['x1', 'y1']])
df['kmeans_5'] = km.labels_

In [42]:
df.drop(columns=['x1', 'y1', 'x2', 'y2'], inplace=True)
df.head()

,los,Provider,female,age,comI10,comI20,comI25,comI48,comI60,comJ44,comE10,comN18,comE66,comI21,comI05,comI34,comI42,comN17,comC00,comI60_9,comF00,comU07,kmeans_5
0,22,"СПб ГБУЗ ""Госпиталь для ветеранов войн""",0.0,82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,"СПб ГБУЗ ""Городская Покровская больница""",0.0,94,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,17,"СПб ГБУЗ ""Госпиталь для ветеранов войн""",1.0,90,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,3
3,14,"СПб ГБУЗ ""Городская Покровская больница""",1.0,74,1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,2
4,5,"СПб ГБУЗ ""Городская Покровская больница""",0.0,77,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
# df.to_excel('clusters.xlsx')